In [12]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [9]:
df1=pd.read_csv("reviews_det.csv")
df1 = df1.rename(columns={'id': 'review_id'})
df1=df1.rename(columns={'listing_id': 'id'})
df1.head()
df1.shape

(431830, 6)

In [10]:
df2=pd.read_csv("listing_det.csv")
df2.head()
df2.shape

C:\Users\nehab\anaconda3\envs\test_torch\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (87) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(20030, 96)

In [7]:
df=pd.merge(df1, df2, on='id', how='left')
df.head()


,id,review_id,date,reviewer_id,reviewer_name,comments,listing_url,scrape_id,last_scraped,name,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,2818,1191,3/30/2009,10952,Lam,Daniel is really cool. The place was nice and ...,https://www.airbnb.com/rooms/2818,2.018120e+13,12/6/2018,Quiet Garden View Room & Super Fast WiFi,...,f,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.1
1,2818,1771,4/24/2009,12798,Alice,Daniel is the most amazing host! His place is ...,https://www.airbnb.com/rooms/2818,2.018120e+13,12/6/2018,Quiet Garden View Room & Super Fast WiFi,...,f,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.1
2,2818,1989,5/3/2009,11869,Natalja,We had such a great time in Amsterdam. Daniel ...,https://www.airbnb.com/rooms/2818,2.018120e+13,12/6/2018,Quiet Garden View Room & Super Fast WiFi,...,f,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.1
3,2818,2797,5/18/2009,14064,Enrique,Very professional operation. Room is very clea...,https://www.airbnb.com/rooms/2818,2.018120e+13,12/6/2018,Quiet Garden View Room & Super Fast WiFi,...,f,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.1
4,2818,3151,5/25/2009,17977,Sherwin,Daniel is highly recommended. He provided all...,https://www.airbnb.com/rooms/2818,2.018120e+13,12/6/2018,Quiet Garden View Room & Super Fast WiFi,...,f,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.1


In [11]:
df.shape

(431830, 101)

In [13]:
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\nehab\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [14]:
def get_rating(review):
    # Compute the sentiment score using Vader
    sentiment = sia.polarity_scores(review)['compound']
    
    # Convert the sentiment score to a rating from 1 to 5
    if sentiment >= 0.9:
        rating = 5
    elif sentiment >= 0.5:
        rating = 4
    elif sentiment >= 0.1:
        rating = 3
    elif sentiment >= -0.5:
        rating = 2
    else:
        rating = 1
    
    return rating

In [16]:
df['ratings'] = df['comments'].astype(str).apply(get_rating)

In [17]:
df.head()

,id,review_id,date,reviewer_id,reviewer_name,comments,listing_url,scrape_id,last_scraped,name,...,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,ratings
0,2818,1191,3/30/2009,10952,Lam,Daniel is really cool. The place was nice and ...,https://www.airbnb.com/rooms/2818,2.018120e+13,12/6/2018,Quiet Garden View Room & Super Fast WiFi,...,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.1,4
1,2818,1771,4/24/2009,12798,Alice,Daniel is the most amazing host! His place is ...,https://www.airbnb.com/rooms/2818,2.018120e+13,12/6/2018,Quiet Garden View Room & Super Fast WiFi,...,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.1,5
2,2818,1989,5/3/2009,11869,Natalja,We had such a great time in Amsterdam. Daniel ...,https://www.airbnb.com/rooms/2818,2.018120e+13,12/6/2018,Quiet Garden View Room & Super Fast WiFi,...,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.1,5
3,2818,2797,5/18/2009,14064,Enrique,Very professional operation. Room is very clea...,https://www.airbnb.com/rooms/2818,2.018120e+13,12/6/2018,Quiet Garden View Room & Super Fast WiFi,...,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.1,5
4,2818,3151,5/25/2009,17977,Sherwin,Daniel is highly recommended. He provided all...,https://www.airbnb.com/rooms/2818,2.018120e+13,12/6/2018,Quiet Garden View Room & Super Fast WiFi,...,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.1,5


In [18]:
rating_counts = df['ratings'].value_counts()

# Print the results
print(rating_counts)

5    230567
4    109011
2     57833
3     18471
1     15948
Name: ratings, dtype: int64


In [19]:
df.to_csv('combined_data.csv', index=False)